# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770386686395                   -0.52    9.0    203ms
  2   -2.771691098681       -2.88       -1.33    1.0    118ms
  3   -2.771714219348       -4.64       -2.42    1.0    102ms
  4   -2.771714633107       -6.38       -3.11    1.0    102ms
  5   -2.771714715188       -7.09       -4.74    2.0    124ms
  6   -2.771714715235      -10.32       -4.78    2.0    121ms
  7   -2.771714715249      -10.87       -5.71    1.0    112ms
  8   -2.771714715250      -12.03       -6.50    2.0    127ms
  9   -2.771714715250      -13.52       -7.25    1.0    111ms
 10   -2.771714715250      -14.65       -7.82    2.0    127ms
 11   -2.771714715250      -14.51       -8.57    1.0    115ms


-0.00013457391718931994

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770483675733                   -0.52    9.0    173ms
  2   -2.771778246521       -2.89       -1.32    1.0    122ms
  3   -2.771801741696       -4.63       -2.46    1.0    121ms
  4   -2.771802000761       -6.59       -3.15    1.0    105ms
  5   -2.771802072856       -7.14       -3.89    2.0    120ms
  6   -2.771802074182       -8.88       -4.35    1.0    182ms
  7   -2.771802074473       -9.54       -5.52    1.0    885ms
  8   -2.771802074476      -11.51       -5.72    1.0    107ms
  9   -2.771802074476      -12.59       -6.52    1.0    107ms
 10   -2.771802074476      -13.51       -7.56    2.0    158ms
 11   -2.771802074476   +  -14.51       -8.44    1.0    145ms


0.017612220157046693

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457391718931994
Displaced dipole:  0.017612220157046693
Polarizability :   1.7746794074236012


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  60.4ms  Non-interacting
   1        0       1       -0.60     10.0   1.16s  
   2        0       2       -2.42      8.0   110ms  
   3        0       3       -3.54      5.0   123ms  
   4        0       4       -5.33      4.0  97.5ms  
   5        0       5       -7.38      1.0  82.8ms  
   6        0       6      -10.31      1.0  86.0ms  
   7        1       1       -7.37     13.0   222ms  Restart
   8        1       2       -8.91      1.0  88.5ms  
                                      13.0   123ms  Final orbitals


(δψ = Matrix{ComplexF64}[[0.002820963452562231 - 0.0017044306202770793im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.15777129596862263 - 0.26614735205081846im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; -0.021094470115619306 - 0.03523308347879071im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.04212122691910139 + 0.0691143553545499im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.6206941122932813e-7 -2.5020151984869706e-7 … -4.880641467195324e-8 -2.5020145804263757e-7; -6.293616270192952e-7 -4.793036293610204e-7 … -1.2077842512015666e-7 -4.793035866611864e-7; … ; 1.3626260352924556e-7 1.1356228906593782e-7 … 4.7119911562746776e-8 1.1356230139314402e-7; 1.3305475157675378e-7 1.363263739345382e-7 … 5.2006867510239344e-8 1.3632642104240108e-7;;; -2.5020146205929327e-7 -1.740490644235747e-7 … -3.5564926490878414e-8 -1.7404901969852561e-7; -4.793035876788432e-7 -3.6797317485968005e-7 … -1.1202892996493846e-7 -3.6797314404328656e-7; … ; 1.135623000184561e-7 1.1211097913838994e-7 … 1.0980684807417144e-7 1

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125255539782
Interacting polarizability:     1.7736548560120862


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 7.74e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 7.06e-11
┌ Info: GMRES linsolve converged at iteration 2, step 2:
│ * norm of residual = 3.96e-12
└ * number of operations = 11
Non-interacting polarizability: 1.9257125255538807
Interacting polarizability:     1.773654851520476


We obtain the identical result to above.